ANALISANDO DADOS DO Uber COM Spark

1- Quantos são e quais são as bases de carros do Uber (onde os carros ficam esperando passageiros)?

2- Qual o total de veículos que passaram pela base B02617?

3- Qual o total de corridas por base? Apresente de forma decrescente.

In [1]:
from pandas import read_csv

In [2]:
#Criando um objeto pandas

uberFile = read_csv('data/uber.csv')

In [3]:
type(uberFile)

pandas.core.frame.DataFrame

In [4]:
#Visualizar as 5 primeiras linhas

uberFile.head(5)

,dispatching_base_number,date,active_vehicles,trips
0,B02512,1/1/2015,190,1132
1,B02765,1/1/2015,225,1765
2,B02764,1/1/2015,3427,29421
3,B02682,1/1/2015,945,7679
4,B02617,1/1/2015,1228,9537


In [5]:
# Transformar o datafram e(pandas) em um Dataframe(Spark)

uberDF = sqlContext.createDataFrame(uberFile)

In [6]:
type(uberDF)

pyspark.sql.dataframe.DataFrame

In [7]:
#Agora vamos criar uma RDD

uberRDD = sc.textFile('data/uber.csv')

In [8]:
type(uberRDD)

pyspark.rdd.RDD

In [9]:
#Vamos pegar o total de registro
uberRDD.count()

355

In [10]:
#Pegar o primeiro registro dos 355
uberRDD.first()

'dispatching_base_number,date,active_vehicles,trips'

In [12]:
#Dividir o arquivo em colunas, separados pelo caracter ","

uberLinhas = uberRDD.map(lambda line: line.split(','))

In [15]:
#Numero de bases de carros uber

uberLinhas.map(lambda linha : linha[0]).distinct().count() - 1

6

In [17]:
#Bases de carros Uber

uberLinhas.map(lambda linha: linha[0]).distinct().collect()

['dispatching_base_number',
 'B02765',
 'B02682',
 'B02598',
 'B02512',
 'B02764',
 'B02617']

In [19]:
#Total de veiculos qu epassaram pela base B02617

uberLinhas.filter(lambda linha: 'B02617' in linha).count()

59

In [20]:
#Agora vamos gravar os dados dos veiculos da base B02617 em um novo RDD

b02617_RDD = uberLinhas.filter(lambda linha: 'B02617' in linha)

#Criamos um novo dataframe

In [22]:
#Total de dias em que o  numero de corridas foi superior a 16.000

b02617_RDD.filter(lambda linha: int(linha[3]) > 16000).count()

4

In [23]:
#Vamos descobrir o total de corridas foi superior a 16.000

b02617_RDD.filter(lambda linha: int(linha[3]) > 16000).collect()

[['B02617', '2/13/2015', '1590', '16996'],
 ['B02617', '2/14/2015', '1486', '16999'],
 ['B02617', '2/20/2015', '1574', '16856'],
 ['B02617', '2/21/2015', '1443', '16098']]

In [24]:
#Vamos criar uam nova RDD
uberRDD2 = sc.textFile('data/uber.csv').filter(lambda line: 'base' not in line).map(lambda line:line.split(','))

In [25]:
#Aplicando redução par acalcular o total por base
uberRDD2.map(lambda kp:(kp[0], int(kp[3]))).reduceByKey(lambda k,v: k + v).collect()

[('B02765', 193670),
 ('B02682', 662509),
 ('B02598', 540791),
 ('B02512', 93786),
 ('B02764', 1914449),
 ('B02617', 725025)]

In [26]:
#Aplicando redução para calcular o total por base, em ordem decrescente

uberRDD2.map(lambda kp: (kp[0], int(kp[3]))).reduceByKey(lambda k, v: k + v).takeOrdered(10, key =lambda x: -x[1])

[('B02764', 1914449),
 ('B02617', 725025),
 ('B02682', 662509),
 ('B02598', 540791),
 ('B02765', 193670),
 ('B02512', 93786)]